In [178]:
import pandas as pd
import random
import numpy as np

#goal: generate SQL code for inserting into database
#there are ways to insert directly from database but not sure if that's allowed

In [179]:
filename = "C:/Projects/3241-project/Proj Data CSV.csv"
filename1 = "C:/Projects/3241-project/Proj Data Unix.csv"

In [180]:
#create dataframe from csv
df = pd.read_csv(filename1, encoding="latin-1")

In [181]:
#fill na values (multiple author books) with the information from the book that they describe
df=df.fillna(method = 'ffill')

In [182]:
#select subset of columns wanted for the table and drop duplicate rows and na; then rename columns
books = df[['ISBN','Title', 'Year', 'Price', 'Publisher']].drop_duplicates()
books.dropna(inplace=True)
books.rename(columns={'Publisher' : 'CompanyName'},inplace=True)
#give value to name attribute (for loop later)
books.name = 'Book'

#TODO: is format really required? it seems like it could possibly create some complicated edge cases in the warehouse (e.g. book not being in stock because it's an ebook)

In [183]:
genres = df[['ISBN', 'Category']].drop_duplicates()
genres.dropna(inplace=True)
genres.rename(columns={'ISBN' : 'BookISBN', 'Category' : 'BookGenre'},inplace=True)
genres.name = 'Genre'

In [184]:
#read in base csv file with names
customer_file = "C:/Projects/3241-project/customers.csv"
customer_names = pd.read_csv(customer_file)

address_file = "C:/Projects/3241-project/addresses.csv"
addresses = pd.read_csv(address_file)

customer = pd.DataFrame(columns=['AccountNumber','AccountBalance','CFirstName', 'CLastName', 'StreetAddress', 'City', 'State', 'Zip', 'Email'])

#assign columns
customer['CFirstName'] = customer_names['CFirstName']
customer['CLastName'] = customer_names['CLastName']
customer['StreetAddress'] = addresses['StreetAddress']

#randomly generate cities, states, and zips and assign to each customer
cities_and_states = [('Columbus','OH',43210),('Cincinnati','OH',43220),('Los Angeles','CA',90210),('New York','NY',10021)]
city_st_zip = [random.choice(cities_and_states) for x in range(len(customer))]
customer['City'] = [city_st_zip[x][0] for x in range(len(customer))]
customer['State'] = [city_st_zip[x][1] for x in range(len(customer))]
customer['Zip'] = [city_st_zip[x][2] for x in range(len(customer))]

#assign ids and generate account balance
customer['AccountNumber'] = [x for x in range(len(customer))]
customer['AccountBalance'] = [round(random.uniform(10,200), 2) for x in range(len(customer))]

#generate emails
last_lower = [customer['CLastName'][x].lower() for x in range(len(customer))]
customer['Email'] = [(lname + "." + str(random.randint(1,30)) + "@gmail.com") for lname in last_lower]

customer.name = 'Customer'

In [185]:
#randomly generate info for each order
order = pd.DataFrame(columns=['ID', 'ShippingAddress', 'TrackingNumber', 'DatePurchased', 'AccountNumber', 'CardNumber'])
num_entries_order = 50
order['ID'] = [x for x in range(num_entries_order)]
order['ShippingAddress'] = [random.choice(addresses['StreetAddress']) for x in range(num_entries_order)]
order['TrackingNumber'] = [x for x in range(num_entries_order)]
#date purchased is in format YYYY-MM-DD
order['DatePurchased'] = ["2023-"+str(random.randint(1, 12))+"-"+ str(random.randint(1, 30))
                          for x in range(num_entries_order)]
#account number is random selection from range of customer ids, which start at 0
order['AccountNumber'] = [random.randint(0,len(customer)) for x in range(num_entries_order)]

order.name = 'Order'
#TODO
#ordertotal is derived attribute so not needed in table; need to revise that later
#what exactly is orderbalance?

#if decide to add, add 'OrderTotal', 'OrderBalance' to columms in correct place

In [186]:
order.head()

,ID,ShippingAddress,TrackingNumber,DatePurchased,AccountNumber,CardNumber
0,0,236 West 11th Avenue,0,2023-3-27,15,NaN
1,1,1810 Cannon Dr,1,2023-7-6,7,NaN
2,2,1810 Cannon Dr,2,2023-10-14,1,NaN
3,3,1858 Neil Ave,3,2023-3-22,19,NaN
4,4,93 West 12th Avenue,4,2023-9-10,10,NaN


In [187]:
#generates random card information to be associated with each account
#this does not take cases with +1 cards/account into consideration, but card number is extra functionality anyway
card = pd.DataFrame(columns=['Number', 'CardCompany', 'CVV', 'NameOnCard', 'ExpirationDate', 'Type', 'StreetAddress', 'City', 'State', 'Zip', 'AccountNumber'])
num_entries_card = len(customer)
card['Number'] = [random.randint(1000000000000000,10000000000000000) for x in range(num_entries_card)]
card['CardCompany'] = [random.choice(['Discover','Capitol One', 'VISA', 'MasterCard']) for x in range(num_entries_card)]
card['NameOnCard'] = customer['CFirstName'] + ' ' + customer['CLastName']
card['CVV'] = [random.randint(100, 999) for x in range(num_entries_card)]
card['ExpirationDate'] = [(str(random.randint(1, 13))+"/"+ str(random.randint(23, 33)))
                            for x in range(num_entries_card)]
card['Type'] = [random.choice(['Debit','Credit']) for x in range(num_entries_card)]
card['StreetAddress'] = customer['StreetAddress']
card['City'] = customer['City']
card['State'] = customer['State']
card['Zip'] = customer['Zip']
card['AccountNumber'] = customer['AccountNumber']

card.name = 'Card'

#find card number associated with order
#TODO: this card number will not necessarily be connected to the account
order['CardNumber'] = [random.choice(card['Number'].to_list()) for x in range(num_entries_order)]

In [188]:
warehouse = pd.DataFrame()
#create based on the cities and states we've already defined
warehouse['City'] = [x[0] for x in cities_and_states]
warehouse['State'] = [x[1] for x in cities_and_states]

warehouse.name = 'Warehouse'

In [189]:
publisher = df[['Publisher']].drop_duplicates()
publisher.dropna(inplace=True)
publisher['Supplier'] = [random.choice(['Supplier1','Supplier2','Supplier3']) for x in range(len(publisher))]

publisher.name = 'Publisher'

In [190]:
#function that splits author into first, middle, last names
#param: full author name string; returns tuple of strings (fname, mname, lname)
def split(authorName):
    names = authorName.split(' ')
    fname = '' ; mname = '' ; lname = ''
    if len(names) == 2:
        fname = names[0]
        lname = names[1]
    elif len(names) == 3:
        fname = names[0]
        mname = names[1]
        lname = names[2]
    elif len(names) == 4:
        fname = names[0]
        mname = names[1] + ' ' + names[2]
        lname = names[3]
    return((fname,mname,lname))

In [191]:
#split authors by spaces
authors = pd.DataFrame()
authors['Author(s)'] = df[['Author(s)']]
authors.drop_duplicates(inplace=True)
authors.dropna(inplace=True)
author_tuples = [x.strip('\"') for x in df['Author(s)']]
author_tuples = [x.strip() for x in author_tuples]
author_tuples = [split(x) for x in author_tuples]

In [192]:
#create new dataframe based on this
authors['AFname']=pd.Series([x[0] for x in author_tuples])
authors['AMname']=pd.Series([x[1] for x in author_tuples])
authors['ALname']=pd.Series([x[2] for x in author_tuples])
#format authors dataframe
authors.drop_duplicates(inplace=True)
authors.dropna(inplace=True)
authors_export = authors[['AFname','AMname','ALname']]

authors.name = 'Author'

In [193]:
ratings = pd.DataFrame(columns=['AccountNumber', 'Rating', 'ISBN'])
num_ratings = 50
ratings['AccountNumber'] = [random.choice(customer['AccountNumber'].to_list()) for x in range(num_ratings)]
ratings['Rating'] = [round(random.uniform(1,5), 1) for x in range(num_ratings)]
ratings['ISBN'] = [random.choice(books['ISBN'].to_list()) for x in range(num_ratings)]

ratings.name = 'Rating'

In [194]:
#join the books and authors dataframe
written_by = df[['ISBN','Author(s)']]
written_by=written_by.join(authors.set_index("Author(s)"), on="Author(s)")
written_by=written_by[['ISBN','AFname','AMname','ALname']]

written_by.name = 'WrittenBy'

In [195]:
order_contains = pd.DataFrame(columns=['ID', 'ISBN', 'NumberOfBooks', 'OrderDate'])
#loop through orders
#note: try to avoid for loops in python if possible but i am too lazy to figure out how to do this in a more pythonic way
for i in range(0,len(order)):
    #choose random number for the number of titles in the order
    num_books_in_order = random.randint(1,5)
    id = order.iloc[i]['ID']
    date=order.iloc[i]['DatePurchased']
    #choose book titles and quantities of each title
    for book in range(0,num_books_in_order):
        isbn = random.choice(books['ISBN'].to_list())
        num_books = random.randint(1,5)
        order_contains.loc[len(order_contains.index)] = [id,isbn,num_books,date]

order_contains.name = 'OrderContains'

In [196]:
stored_in = pd.DataFrame(columns=['ISBN', 'City', 'State', 'NumberOfBooks'])
stored_in_length = 25
stored_in['ISBN'] = [random.choice(books['ISBN'].to_list()) for x in range(stored_in_length)]
city_st = [random.choice(cities_and_states) for x in range(stored_in_length)]
stored_in['City'] = [city_st[x][0] for x in range(stored_in_length)]
stored_in['State'] = [city_st[x][1] for x in range(stored_in_length)]
stored_in['NumberOfBooks'] = [random.randint(0,50) for x in range(stored_in_length)]

stored_in.name = 'StoredIn'

In [197]:
#close stream. just like software!
output_filename = "C:/Projects/3241-project/output.txt"

#create output file
output_file = open(output_filename,"w")

In [198]:
def create_query_text(df, output):
    #initial command
    create = 'INSERT INTO ' + df.name + ' VALUES ' 
    output.write(create)

    #loop through rows in dataframe to create
    #generally you want to avoid for loops in python but here we're working with a stream
    #so i think it's kind of unavoidable
    #loop through rows first
    for i in range(len(df)):
        #create string so everything stays on one line
        string_out = '('

        #loop through attributes next
        for col in range(len(df.columns)):
            #print comma if not last
            if(col < len(df.columns)-1):
                string_out += str(df.iloc[i][col]) + ','
            else:
                string_out += str(df.iloc[i][col]) 

        #print comma if not last
        if(i < len(df)-1):
            string_out += '),'
        else:
                string_out += ')' 
        output.write(string_out)

In [199]:
list_of_db = [books, genres, customer, order, card, warehouse, publisher]

for db in list_of_db:
    create_query_text(db,output_file)

list_of_db_2 = [authors, ratings, written_by, order_contains, stored_in]
for db in list_of_db_2:
    create_query_text(db,output_file)


In [200]:
output_file.close()